In [15]:
import socket
import time
from datetime import datetime
import threading
import pyvisa
import numpy as np

In [2]:
'''
Setup communication
'''
# pip install pyvisa, pyvisa-py'
#find ID's with "lsusb"
#then run this as super user
#echo 'SUBSYSTEM=="usb", ACTION=="add", ATTRS{idVendor}=="05e6", ATTRS{idProduct}=="6500" GROUP="tty", MODE="0660"' >> /etc/udev/rules.d/99-com.rules
# sudo usermod -a -G tty pi

rm = pyvisa.ResourceManager()
print(rm.list_resources('?*'))

inst = rm.open_resource('USB0::0x05E6::0x6500::04440503::INSTR')

inst.write('*IDN?')
print(inst.read())

('USB0::0x05E6::0x6500::04440503::INSTR',)
KEITHLEY INSTRUMENTS,MODEL DMM6500,04440503,1.0.04b



In [3]:
inst.write('SENS:FUNC "CURR:DC"')
inst.write('SENS:CURR:DC:RANG 0.001')
inst.write('SENS:CURR:DC:NPLC 1')

21

In [4]:
def query(s):
    inst.write(s)
    # collect return
    out =''
    while True:
        line = inst.read_bytes(1)
        if line == b'1`':
            if len(out)>0:
                break
            continue
        out += line.decode('utf-8')
        if b'\n' in line: # newline marks end of return
            break
    return out

In [5]:
query('READ?')

'-7.002609E-09\n'

In [17]:
rate = 10
samples = 0
def aquire_as_thread():
    global samples
    while keep_alive:
        s = str(datetime.now()) + '\n'
        for i in range(rate):
            s = s+query('READ?')[:-1]
            time.sleep(1/rate)
            if i<rate:
                s = s + ' '
        with open('multimeter_log.txt','a') as fi:
            fi.write(s+'\n')
        samples += 1
        if keep_alive:
            print(f'multimeter {samples}x{rate} datapoints', end = '\r')
        else:
            break
        
            
keep_alive = True 
threading.Thread(target=aquire_as_thread, args=()).start()


In [22]:
# to stop reading thread:
if 0:
    keep_alive = False
    print(f'multimeter {samples}x{rate} datapoints')

In [23]:
import os
def get_last_lines(file, n):
    with open(file, 'rb') as f:
        if os.path.getsize(file) > 1000:
            f.seek(-1000, 2)
        data = f.readlines()
        lines = []
        for i in range(n,0,-1):
            lines.append(data[-i].decode("utf-8"))
        lines = ' / '.join(lines).replace('\r\n', '').strip()
    return lines

In [24]:
get_last_lines('multimeter_log.txt', 2)

'2022-09-09 15:59:53.279061 / -6.032736E-09 -6.057508E-09 -7.927685E-09 -6.206131E-09 -7.133170E-09 -8.099222E-09 -5.411616E-09 -5.671707E-09 -5.907024E-09 -5.956565E-09'